In [54]:
import os
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

# Ensure GPU is visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Check if CUDA is available
print("CUDA is available" if torch.cuda.is_available() else "CUDA is not available")

# Load your dataset
data = pd.read_csv(r'C:\Users\admin\Desktop\SQ\try.csv')

# Preprocess function
def preprocess_text(text):
    return text.lower().replace('\n', ' ').replace('\r', '')

# Apply preprocessing
data['text'] = data['text'].apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
data['target'] = label_encoder.fit_transform(data['target'])
num_labels = len(label_encoder.classes_)

# Split the dataset
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['target'], random_state=42)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_data.reset_index(drop=True))

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/roberta-large-zeroshot-v2.0-c")

# Tokenize the texts
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Rename the target column to labels
train_dataset = train_dataset.rename_column("target", "labels")
test_dataset = test_dataset.rename_column("target", "labels")

# Print the tokenized datasets to check
print("Tokenized Test Dataset Sample:")
print(test_dataset[:5])
print("Tokenized Train Dataset Sample:")
print(train_dataset[:5])

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Print the formatted datasets to verify
print("Formatted Train Dataset Sample:")
print(train_dataset)
print("Formatted Test Dataset Sample:")
print(test_dataset)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/roberta-large-zeroshot-v2.0-c", num_labels=num_labels, ignore_mismatched_sizes=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./saved_model',
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    no_cuda=not torch.cuda.is_available(),  # Use CUDA if available
    report_to=[]  
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Save the model
trainer.save_model('./saved_model')

# Optionally, save the tokenizer as well
tokenizer.save_pretrained('./saved_model')


CUDA is not available


Map: 100%|██████████| 8/8 [00:00<00:00, 1591.99 examples/s]


Tokenized Test Dataset Sample:
{'text': ['def unveils latest tech product', 'abc reports strong financial results for q2', 'abc corp declares bankruptcy after revenue drop', 'abc corp secures debt financing for expansion', 'company xyz launches new esg initiatives'], 'labels': [5, 7, 0, 1, 2], 'input_ids': [[0, 9232, 36685, 5290, 665, 2903, 1152, 2, 1, 1, 1, 1], [0, 36822, 690, 670, 613, 775, 13, 2231, 176, 2, 1, 1], [0, 36822, 44086, 26460, 7388, 71, 903, 1874, 2, 1, 1, 1], [0, 36822, 44086, 15636, 4123, 1126, 5200, 13, 2919, 2, 1, 1], [0, 24233, 3023, 42006, 10158, 92, 2714, 571, 5287, 2, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]}
Tokenized Train Dataset Sample:
{'text': ['xyz ltd declares bankruptcy amidst financial turmoil', 'def inc. secures $30 million loan from bank', 'abc and xyz form strategic alliance', 

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/roberta-large-zeroshot-v2.0-c and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([8, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 T

{'eval_loss': 2.1254987716674805, 'eval_runtime': 0.1965, 'eval_samples_per_second': 40.711, 'eval_steps_per_second': 5.089, 'epoch': 1.0}


...

100%|██████████| 120/120 [04:28<00:00,  2.20s/it]


                               
                                                 
100%|██████████| 120/120 [04:28<00:00,  2.20s/it]

100%|██████████| 120/120 [04:28<00:00,  2.24s/it]


{'eval_loss': 0.6590226888656616, 'eval_runtime': 0.2561, 'eval_samples_per_second': 31.232, 'eval_steps_per_second': 3.904, 'epoch': 30.0}
{'train_runtime': 268.4704, 'train_samples_per_second': 3.352, 'train_steps_per_second': 0.447, 'train_loss': 1.2363861083984375, 'epoch': 30.0}


100%|██████████| 1/1 [00:00<00:00, 1003.42it/s]


Evaluation results: {'eval_loss': 0.6590226888656616, 'eval_runtime': 0.2647, 'eval_samples_per_second': 30.226, 'eval_steps_per_second': 3.778, 'epoch': 30.0}


('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.json',
 './saved_model\\merges.txt',
 './saved_model\\added_tokens.json',
 './saved_model\\tokenizer.json')

In [56]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric

# Ensure GPU is visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Load your test dataset
data = pd.read_csv(r'C:\Users\admin\Desktop\SQ\try.csv')

# Preprocess function
def preprocess_text(text):
    return text.lower().replace('\n', ' ').replace('\r', '')

# Apply preprocessing
data['text'] = data['text'].apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
data['target'] = label_encoder.fit_transform(data['target'])

# Split the dataset
_, test_data = train_test_split(data, test_size=0.5, stratify=data['target'], random_state=42)

# Convert to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_data.reset_index(drop=True))

# Load the saved model and tokenizer
model_path = './saved_model'
tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\admin\Desktop\traini\saved_model")
model = AutoModelForSequenceClassification.from_pretrained(r"C:\Users\admin\Desktop\traini\saved_model")

# Tokenize the texts
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

test_dataset = test_dataset.map(tokenize, batched=True)

# Rename the target column to labels
test_dataset = test_dataset.rename_column("target", "labels")

# Set format for PyTorch
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Define the compute_metrics function
metric = load_metric("accuracy")

def compute_metrics(p):
    predictions, labels = p
    predictions = torch.tensor(predictions)  # Convert predictions to tensor
    predictions = torch.argmax(predictions, dim=1)
    return metric.compute(predictions=predictions, references=torch.tensor(labels))

# Define evaluation arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=8,
    no_cuda=True,
    report_to=[]  # Disabling all reporting integrations including codecarbon
)

# Create a Trainer instance for evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Evaluate the model
results = trainer.evaluate()

# Print the results
print(results)


Map: 100%|██████████| 19/19 [00:00<00:00, 5524.94 examples/s]
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
100%|██████████| 3/3 [00:02<00:00,  1.13it/s]

{'eval_loss': 0.49643397331237793, 'eval_accuracy': 1.0, 'eval_runtime': 2.869, 'eval_samples_per_second': 6.622, 'eval_steps_per_second': 1.046}
